## Parts of a Failed Project

Here you will find three parts of a cryptocurrency trader bot project that was almost 2 years ago (2021)...

In [1]:
from numbers import Number
from typing import Optional, Union, Sequence, Tuple

import pandas as pd
import ta
import yfinance as yf

### src/datasets.py

In [2]:
def fetch_data(ticker: str, start_date: Optional[str] = None, end_date: Optional[str] = None,
               period: Optional[str] = None, interval: Optional[str] = None) -> pd.DataFrame:
    """Downloads data from Yahoo Finance for the given parameters.

    Args:
        ticker: Ticker symbol of the stock to download data for.
          e.g. AAPL for Apple Inc, GOOGL for Google, BTC-USD for Bitcoin.
        start_date: The date that the data should start from.
          Format should be YYYY-MM-DD.
        end_date: The date that the data should end at.
          The format is the same as start date.
        period: Time period of the data to download. Can be '1d', '5d', '1mo', '3mo',
          '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'.
        interval: Interval of the data. Can be '1m', '2m', '5m', '15m', '30m', '60m', '90m',
          '1h', '1d', '5d', '1wk', '1mo', '3mo'. Default is '1d'.

    Returns:
        A Pandas DataFrame with data like the following if the parameters are valid, otherwise
        an empty DataFrame is returned.

                           Open         High          Low        Close    Adj Close       Volume
        Date
        2017-11-09   308.644989   329.451996   307.056000   320.884003   320.884003    893249984
        2017-11-10   320.670990   324.717987   294.541992   299.252991   299.252991    885985984
        2017-11-11   298.585999   319.453003   298.191986   314.681000   314.681000    842300992
        2017-11-12   314.690002   319.153015   298.513000   307.907990   307.907990   1613479936
        2017-11-13   307.024994   328.415009   307.024994   316.716003   316.716003   1041889984
    """
    if interval is None:  # Yahoo Finance give error if interval is not specified
        interval = '1d'

    return yf.download(
        ticker,
        start=start_date,
        end=end_date,
        period=period,
        interval=interval,
    )

In [3]:
def fetch_open_prices(ticker: str, start_date: Optional[str] = None, end_date: Optional[str] = None,
                      period: Optional[str] = None, interval: Optional[str] = None) -> pd.Series:
    """Returns only the open prices of the data.

    Check the documentation of `fetch_data` in this module for more information.
    """
    return fetch_data(
        ticker,
        start_date=start_date,
        end_date=end_date,
        period=period,
        interval=interval,
    )['Open']

In [4]:
def fetch_high_prices(ticker: str, start_date: Optional[str] = None, end_date: Optional[str] = None,
                      period: Optional[str] = None, interval: Optional[str] = None) -> pd.Series:
    """Returns only the high prices of the data.

    Check the documentation of `fetch_data` in this module for more information.
    """
    return fetch_data(
        ticker,
        start_date=start_date,
        end_date=end_date,
        period=period,
        interval=interval,
    )['High']

In [5]:
def fetch_low_prices(ticker: str, start_date: Optional[str] = None, end_date: Optional[str] = None,
                     period: Optional[str] = None, interval: Optional[str] = None) -> pd.Series:
    """Returns only the low prices of the data.

    Check the documentation of `fetch_data` in this module for more information.
    """
    return fetch_data(
        ticker,
        start_date=start_date,
        end_date=end_date,
        period=period,
        interval=interval,
    )['Low']

In [6]:
def fetch_closed_prices(ticker: str, start_date: Optional[str] = None, end_date: Optional[str] = None,
                        period: Optional[str] = None, interval: Optional[str] = None) -> pd.Series:
    """Returns only the closed prices of the data.

    Check the documentation of `fetch_data` in this module for more information.
    """
    return fetch_data(
        ticker,
        start_date=start_date,
        end_date=end_date,
        period=period,
        interval=interval,
    )['Close']

In [7]:
def fetch_adj_closed_prices(ticker: str, start_date: Optional[str] = None, end_date: Optional[str] = None,
                            period: Optional[str] = None, interval: Optional[str] = None) -> pd.Series:
    """Returns only the adjusted closed prices of the data.

    Check the documentation of `fetch_data` in this module for more information.
    """
    return fetch_data(
        ticker,
        start_date=start_date,
        end_date=end_date,
        period=period,
        interval=interval,
    )['Adj Close']

In [8]:
def fetch_market_volumes(ticker: str, start_date: Optional[str] = None, end_date: Optional[str] = None,
                         period: Optional[str] = None, interval: Optional[str] = None) -> pd.Series:
    """Returns only the market volumes of the data.

    Check the documentation of `fetch_data` in this module for more information.
    """
    return fetch_data(
        ticker,
        start_date=start_date,
        end_date=end_date,
        period=period,
        interval=interval,
    )['Volume']

### src/helpers.py

In [9]:
def to_higher_level_data_structure(
    data1: Union[pd.Series, pd.DataFrame, Sequence[Number]],
    data2: Union[pd.Series, pd.DataFrame, Sequence[Number]],
) -> Union[Tuple[pd.Series, pd.Series], Tuple[pd.DataFrame, pd.DataFrame]]:
    """Converts the given data to a higher level data structure."""
    data1 = data1 if isinstance(data1, (pd.Series, pd.DataFrame)) else pd.Series(data1)
    data2 = data2 if isinstance(data2, (pd.Series, pd.DataFrame)) else pd.Series(data2)

    if isinstance(data1, pd.Series) and isinstance(data2, pd.DataFrame):
        data1 = pd.DataFrame(data1)
    elif isinstance(data1, pd.DataFrame) and isinstance(data2, pd.Series):
        data2 = pd.DataFrame(data2)

    return data1, data2

### src/indicators.py

In [10]:
def crossover(indicator1: Union[Sequence[Number], pd.Series],
              indicator2: Union[Sequence[Number], pd.Series],
              at: Optional[Number] = None) -> bool:
    """Checks if the two given indicators cross over at the given point or not.

    Args:
        indicator1: Result of a technical indicator calculation
          as the first argument.
        indicator2: Result of a technical indicator calculation
          as the second argument.
        at: The value to check for crossover. By default, the last value
          will be used. (It's better to use negative indices to get access)

    Returns:
        True if the last value of the first indicator is higher than the
        first value of the second indicator, False otherwise.
    """
    if isinstance(indicator1, Sequence):
        indicator1 = pd.Series(indicator1)
    if isinstance(indicator2, Sequence):
        indicator2 = pd.Series(indicator2)
    at = at if at is not None else -1
    return indicator1.iloc[at] > indicator2.iloc[at]

In [11]:
def sma(close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Simple Moving Average (SMA) for the given data.

    Args:
        close: The close data to calculate the SMA for.
        period: The period of the SMA.

    Returns:
        A Pandas Series containing the SMA for the given data.
    """
    if isinstance(close, Sequence):  # Tuple and list are also a sequence
        close = pd.Series(close)
    return ta.trend.sma_indicator(close, period)

In [12]:
def wma(close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Weighted Moving Average (WMA) for the given data.

    Args:
        close: The close data to calculate the WMA for.
        period: The period of the WMA.

    Returns:
        A Pandas Series containing the WMA for the given data.
    """
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.trend.wma_indicator(close, period)

In [13]:
def macd(close: Union[Sequence[Number], pd.Series], short_period: int,
         long_period: int, signal_period: int) -> pd.Series:
    """Calculates the Moving Average Convergence/Divergence (MACD) for the given data.

    Args:
        close: The close data to calculate the MACD for.
        short_period: The short period of the MACD.
        long_period: The long period of the MACD.
        signal_period: The period to use for the signal line.

    Returns:
        A Pandas Series containing the MACD for the given data.
    """
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.trend.macd(close, short_period, long_period, signal_period)

In [14]:
def adx(high: Union[Sequence[Number], pd.Series], low: Union[Sequence[Number], pd.Series],
        close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Average Directional Movement Index (ADX) for the given data.

    Args:
        high: The high data to calculate the ADX for.
        low: The low data to calculate the ADX for.
        close: The close data to calculate the ADX for.
        period: The period of the ADX.

    Returns:
        A Pandas Series containing the ADX for the given data.
    """
    if isinstance(high, Sequence):
        high = pd.Series(high)
    if isinstance(low, Sequence):
        low = pd.Series(low)
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.trend.adx(high, low, close, period)

In [15]:
def rsi(close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Relative Strength Index (RSI) for the given data.

    Args:
        close: The close data to calculate the RSI for.
        period: The period of the RSI.

    Returns:
        A Pandas Series containing the RSI for the given data.
    """
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.momentum.rsi(close, period)

In [16]:
def stoch(high: Union[Sequence[Number], pd.Series], low: Union[Sequence[Number], pd.Series],
          close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Stochastic Oscillator (STOCH) for the given data.

    Args:
        high: The high data to calculate the STOCH for.
        low: The low data to calculate the STOCH for.
        close: The close data to calculate the STOCH for.
        period: The period of the STOCH.

    Returns:
        A Pandas Series containing the STOCH for the given data.
    """
    if isinstance(high, Sequence):
        high = pd.Series(high)
    if isinstance(low, Sequence):
        low = pd.Series(low)
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.momentum.stoch(high, low, close, period)

In [17]:
def williams_r(high: Union[Sequence[Number], pd.Series], low: Union[Sequence[Number], pd.Series],
               close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Williams %R for the given data.

    Args:
        high: The high data to calculate the Williams %R for.
        low: The low data to calculate the Williams %R for.
        close: The close data to calculate the Williams %R for.
        period: The period of the Williams %R.

    Returns:
        A Pandas Series containing the Williams %R for the given data.
    """
    if isinstance(high, Sequence):
        high = pd.Series(high)
    if isinstance(low, Sequence):
        low = pd.Series(low)
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.momentum.williams_r(high, low, close, period)

In [18]:
def aroon(close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Aroon for the given data.

    Args:
        close: The close data to calculate the Aroon for.
        period: The period of the Aroon.

    Returns:
        A Pandas Series containing the Aroon for the given data.
    """
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.trend.AroonIndicator(close, period).aroon_indicator()

In [19]:
def aroon_up(close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Aroon Up for the given data.

    Args:
        close: The close data to calculate the Aroon Up for.
        period: The period of the Aroon Up.

    Returns:
        A Pandas Series containing the Aroon Up for the given data.
    """
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.trend.AroonIndicator(close, period).aroon_up()

In [20]:
def aroon_down(close: Union[Sequence[Number], pd.Series], period: int) -> pd.Series:
    """Calculates the Aroon Down for the given data.

    Args:
        close: The close data to calculate the Aroon Down for.
        period: The period of the Aroon Down.

    Returns:
        A Pandas Series containing the Aroon Down for the given data.
    """
    if isinstance(close, Sequence):
        close = pd.Series(close)
    return ta.trend.AroonIndicator(close, period).aroon_down()